In [71]:
import tensorflow as tf

In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [73]:
import tflearn


In [74]:
import data_helper


In [75]:
df_clothing = pd.read_json('Clothing_Shoes_and_Jewelry_5.json', lines = True)

In [76]:
df_sports = pd.read_json('Sports_and_Outdoors_5.json', lines = True )

In [77]:
df = pd.concat([df_clothing,df_sports])

In [78]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0000031887,"[0, 0]",5,This is a great tutu and at a really great pri...,"02 12, 2011",A1KLRMWW2FWPL4,"Amazon Customer ""cameramom""",Great tutu- not cheaply made,1297468800
1,0000031887,"[0, 0]",5,I bought this for my 4 yr old daughter for dan...,"01 19, 2013",A2G5TCU2WDFZ65,Amazon Customer,Very Cute!!,1358553600
2,0000031887,"[0, 0]",5,What can I say... my daughters have it in oran...,"01 4, 2013",A1RLQXYNCMWRWN,Carola,I have buy more than one,1357257600
3,0000031887,"[0, 0]",5,"We bought several tutus at once, and they are ...","04 27, 2014",A8U3FAMSJVHS5,Caromcg,"Adorable, Sturdy",1398556800
4,0000031887,"[0, 0]",5,Thank you Halo Heaven great product for Little...,"03 15, 2014",A3GEOILWLK86XM,CJ,Grammy's Angels Love it,1394841600


In [79]:
reviews = df['reviewText'].values

In [80]:
del df

In [81]:
reviews_text = [data_helper.clean_str(sent) for sent in reviews]

In [82]:
max_sent_length = 100

In [83]:
vocab_processor = tflearn.data_utils.VocabularyProcessor(max_sent_length)

In [84]:
data = list(vocab_processor.fit_transform(reviews_text))

In [85]:
vocab_size = len(vocab_processor.vocabulary_)
data = [i.tolist() for i in data]
for lst in data:
    try:
        ##pop off last index if it is equal to 0
        while lst[-1] == 0:
            lst.pop()
    except:
        pass


In [49]:
data = np.array(list(data))

In [50]:
data

array([list([1, 2, 3, 4, 5, 6, 7, 3, 8, 4, 9, 10, 11, 12, 13, 14, 7, 15, 16, 17, 18, 19, 20, 21, 22, 6, 23, 24, 25, 26, 5, 27, 2, 12, 28, 29, 3]),
       list([19, 30, 1, 31, 32, 33, 34, 35, 36, 31, 37, 38, 39, 40, 10, 41, 31, 42, 43, 44, 6, 42, 45, 46, 10, 47, 48, 19, 30, 1, 49, 50, 51, 3, 52, 53, 54, 55, 56, 6, 47, 57, 42, 58, 59, 60, 4, 9, 47, 61, 62, 63, 64, 65, 66, 67, 50, 31, 68, 69, 70, 71]),
       list([72, 73, 19, 74, 32, 75, 76, 10, 77, 78, 79, 80, 6, 81, 6, 19, 82, 83, 49, 84, 31, 85, 42, 86, 87, 10, 2, 3, 61, 62, 88, 31, 89, 3, 90, 91, 4, 58, 92, 93, 4, 94, 49, 95, 96, 6, 97, 98, 99, 10, 19, 100, 10, 2, 3, 4, 84, 31, 101, 6, 102, 63]),
       ...,
       list([1, 168533, 3761, 1647, 6117, 3128, 4322, 2569, 4178, 2, 4, 21, 3, 888, 276, 2051, 873, 12885, 5072, 31, 560, 288, 1940, 435, 146, 325, 65, 4809, 19734, 10, 151, 1920, 250, 1, 171, 151, 292, 4178, 19, 76, 142, 202, 161, 1089, 1278, 19, 76, 193, 10, 4625, 21, 10, 151, 419, 77, 32, 2254, 2847, 2254, 6, 10, 47, 1035, 17,

In [51]:
window = 2

In [52]:
pivot_words = [] #Input Words 

In [53]:
target_words =[] #Output Words

In [59]:
for d in range(data.shape[0]):
    pivot_idx = data[d][window:-window]
    
    for i in range(len(pivot_idx)):
        pivot = pivot_idx[i]
        
        targets = np.array([])
        
        neg_target = data[d][i : window + i]
        pos_target = data[d][i + window + 1 : i + window + window + 1]
        
        targets= np.append(targets,[neg_target,pos_target]).flatten().tolist()
        
        for c in range(window*2):
            pivot_words.append(pivot)
            target_words.append(targets[c])

In [60]:
pivot_words[:10]


[3, 3, 3, 3, 4, 4, 4, 3, 3, 3]

In [62]:
target_words[:15]

[1.0,
 2.0,
 4.0,
 5.0,
 [2, 3],
 [5],
 1.0,
 2.0,
 4.0,
 5.0,
 [2, 3],
 [5],
 1.0,
 2.0,
 4.0]

In [67]:
embedding_size = 128

In [70]:
def word2vec():
    #Pivot words
    x = tf.placeholder(tf.int32,shape=[None,],name="x_pivot_idxs")
    #Target Words
    y = tf.placeholder(tf.int32,shape=[None,],name="y_target_idxs")
    
    
    embedding = tf.Variable(tf.random_uniform([vocab_size,embedding_size],-1.0,1.0),name="word_embedding")
    
    
    nce_weights = tf.Variable(tf.truncated_normal([vocab_size,embedding_size],stddev=tf.sqrt(1/embedding_size)),name="nce_weights")
    
    nce_biases = tf.Variable(tf.zeroes([vocab_size]), name = "nce_biases")
    
    
    #lookup pivot word embedding 
    pivot = tf.nn.embedding_lookup(Embedding, x, name = "word_embed_lookup")
    
    train_labels = tf.reshape(y, [tfshape(y)[0],1])
    